In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [16]:
dtest = pd.read_csv("train.csv")

ftest = pd.read_csv("test.csv")

dtest.drop("Name", axis=1, inplace = True)

In [17]:
y = dtest.Transported
features = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

X = dtest[features]
X2 = ftest[features]

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

In [18]:
OHcols = ["CryoSleep", "VIP", "Destination", "HomePlanet"]

cols2 = ["Cabin", "PassengerId"]
dtest2 = dtest[cols2]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
#Up here is changing the amount of rows
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[OHcols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[OHcols]))
OH_cols_final = pd.DataFrame(OH_encoder.fit_transform(X2[OHcols]))


# Putting index bacl
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_final.index = X2.index

# Removing previous columns
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_final = X2.drop(object_cols, axis = 1)

# Adding to other columns
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_final = pd.concat([num_final, OH_cols_final], axis=1)
# Imputing the numerical columns
my_imputer = SimpleImputer()
Mid_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
Mid_X_valid = pd.DataFrame(my_imputer.transform(OH_X_valid))
Mid_final = pd.DataFrame(my_imputer.transform(OH_X_final))

# Imputation removed column names; put them back
Mid_X_train.columns = OH_X_train.columns
Mid_X_valid.columns = OH_X_valid.columns
Mid_final.columns = OH_X_final.columns


/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', '

In [21]:
model = XGBClassifier()
model.fit(Mid_X_train, y_train)

[10:08:35] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [44]:
preds_valid = model.predict(Mid_X_valid)

# 4 is the best option for max_depth

# for n in [0,1,2,3,4]:
#     model = XGBClassifier(max_depth = 4, max_delta_step = n)
#     model.fit(Mid_X_train, y_train)
#     preds_valid = model.predict(Mid_X_valid)
#     print(n, accuracy_score(y_valid, preds_valid))

from sklearn.ensemble import GradientBoostingClassifier

# for n in [4,5,6,7]:
#     model = GradientBoostingClassifier(max_depth = n)
#     model.fit(Mid_X_train, y_train)
#     preds_valid = model.predict(Mid_X_valid)
#     print(n, accuracy_score(y_valid, preds_valid))


/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [40]:
train_final = pd.concat([Mid_X_valid, Mid_X_train])
y_final = pd.concat([y_valid, y_train])
model.predict(train_final)
y_final.isna().sum()

model_final = XGBClassifier(random_state = 1, max_depth = 4)
model_final.fit(train_final, y_final)
preds = model_final.predict(Mid_final)


/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/sarahbraverman/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[10:31:04] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [41]:
output = pd.DataFrame({"PassengerId": ftest.PassengerId, "Transported": preds})
output.to_csv('submission4.csv', index = False)

1741
Sarah Braverman
0.79027
Your Best Entry!
Your most recent submission scored 0.79027, which is an improvement of your previous score of 0.78185. Great job!

I changed the max_depth parameter 

Your most recent submission scored 0.79167, which is an improvement of your previous score of 0.79027. Great job!
